# AlexNet & CommonLoopUtils

In this example, we will implement [AlexNet](https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf) and also use [CommonLoopUtils](https://github.com/google/CommonLoopUtils) from Google to track the model training.

## Short (and skippable) Introduction

AlexNet is a CNN model from 2012, which arguably brought the "deep" to deep learning. It trained on ImageNet, which is a massive dataset consisting of over 15 million images and over 22000 classes and before AlexNet, that dataset was a tough nut to crack! To solve that dataset, the authors brought a few novelties to the table, some of which we will, skip. Those were:

- Using ReLU (instead of Sigmoid), which boosted performance, speed and addressed the vanishing gradient problem
- Multi GPU training (we will skip this, but they wrote a bunch of custom CUDA code to train the whole thing on TWO whole GPUs! Outrageous at that time)
- Local Response Normalisation (largely superseded by Batch Normalisation these days, but still interesting)
- Overlapping Max Pooling (basically using a stride of 2 instead of 1)
- Dropout to prevent overfitting
- a bunch of data augmentation:
    - crop the image to 256x256
    - select random 224x224 portion of the cropped image (we will do something similar)
    - randomly flip the image horizontally
    - PCA colour augmentation

None of these were solely their invention (except for the normalisation method and the multi GPU stuff) but the combination of all these techniques is what wrote history. 

In this example, though, we won't be training on the ImageNet dataset, but instead on the `cats_vs_dogs` dataset, which is much harder than MNIST, but still small enough that you can train the thing in a relatively short amount of time (~30 minutes on a 3090).